## 简答题(共4题，每题5分，共计20分)

#### 1. 请写出你了解的机器学习特征工程操作，以及它的意义（面试题）

##### 特征工程

- 1.数值幅度缩放:log,MinMax,Z-score
- 2.统计结果作为新特征
- 3.缺失值处理
- 4.高次特征和交叉特征   
   - 通过生成高次项特征增加维度 PolynomialFeatures专门产生多项式的，并且多项式包含的是相互影响的特征集。
   比如：一个输入样本是２维的。形式如[a,b] ,则二阶多项式的特征集如下[1,a,b,a^2,ab,b^2]
- 5.离散化、分箱、分桶   
   - 等距离cut：按等间距切分 等频qcut：百分比切分
- 6.独热向量编码
- 7.时间日期型   
   - 可以得到其他维度的时间特征
- 8.组合特征

##### 特征选择
- 1.过滤式/Filter（较少使用） 根据每一列与target的相关性选择
- 2.包裹式/wrapper RFE,递归去除列评估结果
- 3.嵌入式/Embedded

#### 意义总结：
特征工程是从原始数据中去除噪音、提取有用信息的过程，提取后的特征能够最大限度地反映数据的统计特性，对最终的预测和分类有更强的表现力

#### 2.请写出上述特征工程操作的sklearn或者pandas实现方式

#### 1.幅度缩放
```
df.Age.apply(np.log)
sklearn:MinMaxScaler
sklearn:StandardScaler
```
#### 2.统计结果作为特征：
```
df_train.loc[:,'FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 
df_train.sample(10)
```
#### 3.缺失值处理
```
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
```
#### 4.高次特征和交叉特征
```
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.fit_transform(df_train[['SibSp','Parch']])
```
#### 5.归一化
```
Normalizer()
```
#### 6.独热向量编码

```
pandas.get_dummies()
skelarn:OneHotEncoder
```

#### 7.日期时间
```
pandas:to_datetime
dt.month dt.dayofweek dt.dayofyear
```

#### 8.文本
````
sklearn:CountVectorizer
sklearn:TfidfVectorizer
gensim:word2vec
````


#### 特征选择：
#### 1.过滤式/Filter（较少使用）
根据每一列与target的相关性选择
```
from sklearn.feature_selection import SelectPercentile
import numpy as np
select = SelectPercentile(percentile=70)
```
#### 2.包裹式/wrapper
RFE,递归去除列评估结果
```
from sklearn.feature_selection import RFE
select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=40)
#select = RFE(LogisticRegression(penalty="l1"), n_features_to_select=40)
```

#### 3.嵌入式/Embedded
```
from sklearn.feature_selection import SelectFromModel
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")
```

#### 3.模型评估中的留一法，留出法，交叉验证分别是什么操作？

#### 留一法：
就是每次只留下一个样本做测试集，其它样本做训练集，如果有k个样本，则需要训练k次，测试k次。

#### 留出法：
把D划分为两部分：训练集S和测试集T，在S上训练，在T上做测试。

#### 交叉验证:
将训练集划分为K折，取一部分作为测试集，其他K-1部分作为训练集，对训练集训练后，然后切换训练集-测试集，从K-1的部分中选取一个测试机，剩下的作为训练集，然后在训练模型，这样循环完毕K折，得到的结果求平均。

#### 步骤：
1、 将全部训练集 S分成 k个不相交的子集，假设 S中的训练样例个数为 m，那么每一个子 集有 m/k 个训练样例，，相应的子集称作 {s1,s2,…,sk}。   
2、每次从分好的子集中里面，拿出一个作为测试集，其它k-1个作为训练集   
3、根据训练训练出模型或者假设函数。   
4、 把这个模型放到测试集上，得到分类率。   
5、计算k次求得的分类率的平均值，作为该模型或者假设函数的真实分类率。 

#### 4.如何理解模型的过拟合与欠拟合，以及如何解决？（面试题）

#### 欠拟合：
模型表达能力弱，不能完全拟合数据集。
#### 解决欠拟合：
增加维度特征，可以通过多项式高次特征、统计等方式增加新的维度

#### 过拟合：
模型过于拟合训练集，导致对于一些噪声误差数据也进行了拟合，这样的模型泛化能力较差。说直白就是对训练集进行了”死记硬背“，会导致到了测试集上无法拟合。
#### 解决过拟合:
对某些模型来讲最粗暴的方式就是直接增加训练集的大小   
在代价函数后面添加正则化项（L1、L2正则），例如lasso和ridge等   
bagging方式中的投票机制和bootstrap，例如随机森林等,让每棵树只看到一部分样本   
树模型中可以对树的深度、叶子结点个数等参数进行限制，防止模型过拟合   
树模型的剪枝操作   
神经网络中的dropout等   

## 操作题(共1题，共计80分)

### 信用卡欺诈项目(共7项，前5项每题10分，6，7题每题15分)

 #### 前期数据导入,预览及处理(此部分勿修改，涉及的数据文件无需复制移动)

In [41]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import zipfile
with zipfile.ZipFile('../Data/KaggleCredit2.zip', 'r') as z:
    f = z.open('KaggleCredit2.csv')
    data = pd.read_csv(f, index_col=0)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45.0,2.0,0.802982,9120.0,13.0,0.0,6.0,0.0,2.0
1,0,0.957151,40.0,0.0,0.121876,2600.0,4.0,0.0,0.0,0.0,1.0
2,0,0.658180,38.0,1.0,0.085113,3042.0,2.0,1.0,0.0,0.0,0.0
3,0,0.233810,30.0,0.0,0.036050,3300.0,5.0,0.0,0.0,0.0,0.0
4,0,0.907239,49.0,1.0,0.024926,63588.0,7.0,0.0,1.0,0.0,0.0


In [42]:
data.shape

(112915, 11)

In [43]:
data.isnull().sum(axis=0)

SeriousDlqin2yrs                           0
RevolvingUtilizationOfUnsecuredLines       0
age                                     4267
NumberOfTime30-59DaysPastDueNotWorse       0
DebtRatio                                  0
MonthlyIncome                              0
NumberOfOpenCreditLinesAndLoans            0
NumberOfTimes90DaysLate                    0
NumberRealEstateLoansOrLines               0
NumberOfTime60-89DaysPastDueNotWorse       0
NumberOfDependents                      4267
dtype: int64

In [44]:
data.dropna(inplace=True)
data.shapey = data['SeriousDlqin2yrs']
X = data.drop('SeriousDlqin2yrs', axis=1)

/jxy8/jxy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [45]:
y = data['SeriousDlqin2yrs']
X = data.drop('SeriousDlqin2yrs', axis=1)
y.mean()

0.06742876076872101

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108648 entries, 0 to 112914
Data columns (total 10 columns):
RevolvingUtilizationOfUnsecuredLines    108648 non-null float64
age                                     108648 non-null float64
NumberOfTime30-59DaysPastDueNotWorse    108648 non-null float64
DebtRatio                               108648 non-null float64
MonthlyIncome                           108648 non-null float64
NumberOfOpenCreditLinesAndLoans         108648 non-null float64
NumberOfTimes90DaysLate                 108648 non-null float64
NumberRealEstateLoansOrLines            108648 non-null float64
NumberOfTime60-89DaysPastDueNotWorse    108648 non-null float64
NumberOfDependents                      108648 non-null float64
dtypes: float64(10)
memory usage: 9.1 MB


In [47]:
y.head()

0    1
1    0
2    0
3    0
4    0
Name: SeriousDlqin2yrs, dtype: int64











#### 以下为操作题

- 1.把数据切分成训练集和测试集

In [48]:
from sklearn.model_selection import train_test_split 

train_x,test_x,train_y,test_y  = train_test_split(X,y,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape,)

(76053, 10) (32595, 10) (76053,) (32595,)


- 2.使用logistic regression建模，并且输出一下系数，分析重要度。   

In [49]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_x,train_y)
lr.predict(test_x)
print(lr.get_params)


/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)>


- 3.使用决策树/SVM/KNN...等sklearn分类算法进行分类，尝试了解参数含义，调整不同的参数。

In [50]:
# 决策树
from sklearn.tree import DecisionTreeClassifier
# 参数 
#  criterion：树结点分裂标准，可选Gini/entropy
#  max_depth：树的最大深度
#  random_state：The minimum number of samples required to split an internal node
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(train_x, train_y)

#default CART
tree_Gini = DecisionTreeClassifier(max_depth=5, random_state=0)
tree_Gini.fit(train_x, train_y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [51]:
# SVM
from sklearn.svm import SVC
# C:误差项的惩罚参数c
# kernel:核函数
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(train_x.head(500), train_y.head(500))

svm_rbf = SVC(kernel='rbf', C=2.0, random_state=0)
svm_rbf.fit(train_x.head(500), train_y.head(500)) 


/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [52]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
# n_neighbors ：分组
# p：Power parameter for the Minkowski metric.
# metric：the distance metric to use for the tree.
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(train_x, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

- 4.在测试集上进行预测，计算准确度

In [23]:
from sklearn.metrics import accuracy_score
# 逻辑回归
pred_y_LR = lr.predict(test_x)
print('LR: %.2f' % accuracy_score(test_y, pred_y_LR))

# 决策树
pred_y_DT = tree.predict(test_x)
print('DT: %.2f' % accuracy_score(test_y, pred_y_DT))

# 决策树CART
pred_y_CART = tree_Gini.predict(test_x)
print('CART: %.2f' % accuracy_score(test_y, pred_y_CART))


# SVM
pred_y_SVM = svm.predict(test_x)
print('SVM: %.2f' % accuracy_score(test_y, pred_y_SVM))

pred_y_SVM_rbf = svm_rbf.predict(test_x)
print('svm_rbf: %.2f' % accuracy_score(test_y, pred_y_SVM_rbf))

#KNN
pred_y_KNN = knn.predict(test_x)
print('KNN: %.2f' % accuracy_score(test_y, pred_y_KNN))



LR: 0.93
DT: 0.93
CART: 0.93
SVM: 0.91
svm_rbf: 0.93
KNN: 0.93


- 5.查看sklearn的官方说明，了解混淆矩阵等评估标准，并对此例进行评估。

In [24]:
# 混淆矩阵是预测值与实际值得交叉情况。列代表预测的数据类别，行代表实际的类别
from sklearn.metrics import confusion_matrix
from sklearn import metrics

matrix = confusion_matrix(test_y, pred_y_LR)
TP = matrix[0,0]
FP = matrix[0,1]
FN = matrix[1,0]
TN = matrix[1,1]
print('混淆矩阵：\n',matrix)
# 结果中，30318表示预测为正样本并且实际为正样本的数量。第一行第二列表示预测为负样本，实际为正样本的数量是75
# 第二行第一列表示实际为负样本预测为正样本的数量是2094，第二行第二列表示实际为负样本预测为负样本的数量为108

混淆矩阵：
 [[30320    59]
 [ 2134    82]]


In [25]:
from sklearn.metrics import classification_report
print("LR:\n",classification_report(test_y, pred_y_LR))
print("DT:\n",classification_report(test_y, pred_y_DT))

print("CART:\n",classification_report(test_y, pred_y_CART))

print("SVM:\n",classification_report(test_y, pred_y_SVM))

print("SVM_rbf:\n",classification_report(test_y, pred_y_SVM_rbf))

print("KNN:\n",classification_report(test_y, pred_y_KNN))



LR:
               precision    recall  f1-score   support

           0       0.93      1.00      0.97     30379
           1       0.58      0.04      0.07      2216

    accuracy                           0.93     32595
   macro avg       0.76      0.52      0.52     32595
weighted avg       0.91      0.93      0.90     32595

DT:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97     30379
           1       0.60      0.10      0.17      2216

    accuracy                           0.93     32595
   macro avg       0.77      0.55      0.57     32595
weighted avg       0.92      0.93      0.91     32595

CART:
               precision    recall  f1-score   support

           0       0.94      0.99      0.97     30379
           1       0.55      0.21      0.30      2216

    accuracy                           0.93     32595
   macro avg       0.75      0.60      0.63     32595
weighted avg       0.92      0.93      0.92     32595

SV

- 6.银行通常会有更严格的要求，因为fraud带来的后果通常比较严重，一般我们会调整模型的标准。   
比如在logistic regression当中，一般我们的概率判定边界为0.5，但是我们可以把阈值设定低一些，来提高模型的“敏感度”   
试试看把阈值设定为0.3，再看看这个时候的混淆矩阵等评估指标。

In [28]:
from sklearn.linear_model import LogisticRegression

lr_proba = lr.predict_proba(test_x)
proba = lr_proba[:,1] > 0.3
print("thredhold = 0.3 LR:\n",classification_report(test_y, proba))
lr_proba[:,1]

thredhold = 0.3 LR:
               precision    recall  f1-score   support

           0       0.94      0.99      0.96     30379
           1       0.52      0.09      0.15      2216

    accuracy                           0.93     32595
   macro avg       0.73      0.54      0.56     32595
weighted avg       0.91      0.93      0.91     32595



array([0.10701581, 0.02688398, 0.08725215, ..., 0.01721289, 0.0137342 ,
       0.1119375 ])

In [33]:
tree_proba = tree.predict_proba(test_x)
print(tree_proba)
tree_proba = tree_proba[:,1] > 0.3
print("thredhold = 0.3 DT:\n",classification_report(test_y, tree_proba))


[[0.77944647 0.22055353]
 [0.98247978 0.01752022]
 [0.98247978 0.01752022]
 ...
 [0.98247978 0.01752022]
 [0.98247978 0.01752022]
 [0.98247978 0.01752022]]
thredhold = 0.3 DT:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96     30379
           1       0.47      0.30      0.37      2216

    accuracy                           0.93     32595
   macro avg       0.71      0.64      0.67     32595
weighted avg       0.92      0.93      0.92     32595



- 7.尝试对不同特征的重要度进行排序，通过特征选择的方式，对特征进行筛选。并重新建模，观察此时的模型准确率等评估指标。

In [39]:
# filter 
from sklearn.feature_selection import SelectPercentile
import numpy as np
select = SelectPercentile(percentile=70)
select.fit(train_x, train_y)
# transform training set:
X_train_selected = select.transform(train_x)
print('特征得分：',select.scores_)
print('筛选前的维度：',train_x.shape)
print('筛选后的维度：',X_train_selected.shape)
np.argsort(select.scores_)

特征得分： [3.70073112e-01 8.72174369e+02 1.22526212e+03 2.56484919e+02
 3.71053102e+01 9.48652301e+01 1.02283377e+03 3.88522040e+01
 7.36301511e+02 1.71194067e+02]
筛选前的维度： (76053, 10)
筛选后的维度： (76053, 7)


array([0, 4, 7, 5, 9, 3, 8, 1, 6, 2])

In [40]:
# 基于以上筛选，对筛选后的特征进行建模并评估
from sklearn.linear_model import LogisticRegression

# 整理测试集，手动过滤特征集0, 4, 7, 5, 9, 3, 8
X_test_selected = test_x.drop([test_x.columns[6],test_x.columns[1],test_x.columns[2]],axis=1)
    
lr_selected = LogisticRegression()

lr_selected.fit(X_train_selected,train_y)
pred = lr_selected.predict(X_test_selected)
print("LR:\n",classification_report(test_y, pred))
matrix2 = confusion_matrix(test_y, pred)
matrix2

/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00     30379
           1       0.07      1.00      0.13      2216

    accuracy                           0.07     32595
   macro avg       0.53      0.50      0.06     32595
weighted avg       0.94      0.07      0.01     32595



array([[   11, 30368],
       [    0,  2216]])